# Geomechanical Injection Scenario Toolkit (GIST)

#Disclaimer
GIST aims to give the _gist_ of a wide range of potential scenarios and aid collective decision making when responding to seismicity.

The results of GIST are entirely dependent upon the inputs provided, which may be incomplete or inaccurate.

There are other potentially plausible inducement scenarios that are not considered, including fluid migration into the basement, 
out-of-zone poroelastic stressing, or hydraulic fracturing.

None of the individual models produced by GIST accurately represent what happens in the subsurface and cannot be credibly used 
to accurately assign liability or responsibility for seismicity.

"All models are wrong, but some are useful" - George Box, 1976

## Prerequisites

Assumes InjectionSQLScheduled completed successfully and injection data are sampled uniformly in time

##Install Dependencies
- geopandas
- gistMC.py
- eqSQL.py
- gistPlots.py
- numpy
- scipy
- pandas


In [0]:
%restart_python

In [0]:
%run "/Workspace/_utils/Utility_Functions"

In [0]:
!pip install geopandas
!pip install geodatasets
!pip install contextily
#! pip install folium matplotlib mapclassify contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 141.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

##Paths

In [0]:
# Paths
homePath='/Workspace/Users/bill.curry@exxonmobil.com/'
# Injection data path 
injPath=homePath+'injection/WeeklyRun/ScheduledOutput/'
# GIST library path
gistPath=homePath+'GIST/'

##Libraries

- numpy
- scipy
- pandas
- matplotlib
- geopandas
- pyspark


In [0]:
import sys
sys.path.append(gistPath+'lib')

In [0]:
#Databricks-specific
#import dataBricksConfig as db
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("eqSQL").getOrCreate()

In [0]:
import numpy as np
import pandas as pd
import os
import gistMC as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
#import contextily as cx

In [0]:
import gistPlots as gp

#1. Select Event

In [0]:
eventID='texnet2025agkw'
forecastYears=5.

##1.1 Create directories, fetch EQ

In [0]:
runPath=gistPath+'/runs/'+eventID+'/'
os.makedirs(runPath, exist_ok=True)

In [0]:
eqs=es.eqSQL()
EQDF=eqs.getEarthquake(eventID)
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# What about the other fault plane solution? - 251 / 36 / -97
EQDF['Strike']=80.
EQDF['Dip']=55.
EQDF['Rake']=-85.
# We could make use of the Rake
EQDF.info()
eq=EQDF.loc[0]

getEarthquake:     SeismicEventId  ... B3RecordDeletedUTCDateTime
0         2743652  ...                        NaT

[1 rows x 32 columns] 1
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              1 non-null      int64         
 1   DataSource                  1 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     1 non-null      object        
 4   EventTimeUtc                1 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    1 non-null      object        
 6   EventTimeZone               1 non-null      object        
 7   EventType                   1 non-null      object        
 8   DepthKm                     1 non-null      float64       
 9   DepthErrorKm                1 non-null   

In [0]:
EQDF.to_csv(runPath+'EQ.csv')

#2. Initial Run

##2.1 Subsurface Model Parameters

###Inputs:

In [0]:
# Binary deep/shallow parameter
deepOrShallow='Deep'
# Depth from surface (ft)
# Distinguishes deep/shallow where we don't have a horizon 
depthCutoff=8000.

In [0]:
nRealizations=200

In [0]:
# Porosity in percent
PorosityPercentMin=3.
PorosityPercentMax=15.

# Permeability in millidarcies
PermMDMin=3.
PermMDMax=1000.

# Thickness in feet
ThicknessFTMin=700.
ThicknessFTMax=2000.

# Vertical compressibility minimum/maximum (1/Pa)
VerticalCompressibilityMin=0.00000000107
VerticalCompressibilityMax=0.00000000005

In [0]:
# Verbosity - 0=silent, 1=some, 2=lots
verb=2
# Minimum Pressure change to care about in PSI
dPCutoff=1.
# Maximum Number of wells to plot
nWells=50
# What is the first year we plot?
minYear=1980

In [0]:
# Water density minimum/maximum (kg/m3)
WaterDensityMin=1000.
WaterDensityMax=1050.
# Water viscosity minimum/maximum (Pa.s)
WaterViscosityMin=0.000799
WaterViscosityMax=0.000801
# Fluid Compressibility minimum/maximum (1/Pa)
FluidCompressibilityMin=0.000000000359
FluidCompressibilityMax=0.000000000361

### Set Interval-Specific Paths...


In [0]:
# Set an output directory for this earthquake and this interval
runIntervalPath=runPath+deepOrShallow+'/'
initialRunIntervalPath=runIntervalPath+'initialRun/'
updatedRunIntervalPath=runIntervalPath+'udpatedRun/'
forecastRunIntervalPath=runIntervalPath+'forecastRun/'
disposalPath=runIntervalPath+'updatedDisposal/'
# Make directory if it doesn't exist
os.makedirs(runIntervalPath, exist_ok=True)
os.makedirs(initialRunIntervalPath, exist_ok=True)
os.makedirs(updatedRunIntervalPath, exist_ok=True)
os.makedirs(forecastRunIntervalPath, exist_ok=True)
os.makedirs(disposalPath, exist_ok=True)
os.makedirs(initialRunIntervalPath+'perWell', exist_ok=True)
os.makedirs(updatedRunIntervalPath+'perWell', exist_ok=True)
os.makedirs(forecastRunIntervalPath+'perWell', exist_ok=True)
################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runIntervalPath+'MC'

In [0]:
# Point to appropriate well and injection files from initial database
if deepOrShallow=='Deep':
  WellFile=injPath+'/deep.csv'
  InjFile=injPath+'/deepReg.csv'
elif deepOrShallow=='Shallow':
  WellFile=injPath+'/shallow.csv'
  InjFile=injPath+'/shallowReg.csv'

In [0]:
# Oversampling of time axis (poroelastic-only)
nTimeBins=21
# Poroelastic parameters - for in-zone poroelasticity in v2
ShearModulusMin=4e9
ShearModulusMax=6e9
PoissonsRatioDrainedMin=0.295
PoissonsRatioDrainedMax=0.305
PoissonsRatioUndrainedMin=0.305
PoissonsRatioUndrainedMax=0.315
BiotsCoefficientMin=0.26
BiotsCoefficientMax=0.36
# Fault parameters (poroelastic-only)
FaultFrictionCoeffMin=0.55
FaultFrictionCoeffMax=0.65
RockFrictionCoeffMin=0.55
RockFrictionCoeffMax=0.65

##2.2 Results

###Compute

In [0]:
gist=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
gist.initPP(rho0_min=WaterDensityMin,
             rho0_max=WaterDensityMax,
             phi_min=PorosityPercentMin,
             phi_max=PorosityPercentMax,
             kMD_min=PermMDMin,
             kMD_max=PermMDMax,
             h_min=ThicknessFTMin,
             h_max=ThicknessFTMax,
             alphav_min=VerticalCompressibilityMin,
             alphav_max=VerticalCompressibilityMax,
             beta_min=FluidCompressibilityMin,
             beta_max=FluidCompressibilityMax)

In [0]:
gist.addWells(wellFile=WellFile,injFile=InjFile,userWellFile=None,userInjFile=None,verbose=verb)
if verb>0:
  gist.wellDF.info()
  EQDF.info()

 gistMC.addWells: no user wells and injection provided, only default wells/injection
 gistMC.addWells: well file added with  2863  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'UniqueWellIdentifier', 'UICNumber',
       'APINumber', 'LeaseName', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType', 'WellName',
       'WellNumber', 'State', 'Basin', 'County', 'District', 'SRAOrSIR',
       'B3InjectionType', 'B3InjectionStatus', 'RegulatoryInjectionType',
       'PermittedMaxLiquidBPD', 'PermittedMaxLiquidPSIG',
       'PermittedMaxGasMCFPerDay', 'PermittedMaxGasPSIG',
       'PermittedCommercialStatus', 'PermittedIntervalTopFt',
       'PermittedIntervalBottomFt', 'InjectionClass', 'PermitStage',
       'PermittedWellDepthClassification', 'DaysApplicationHasBeenInReview',
       'DaysToPermitApproval', 'PermitIsAmendment',
       'PendingApplicationIsAmendment',
       'PendingApplicationRequestedCommercialStatus',
     

In [0]:
selectedWellsDF,ignoredWellsDF,injDF=gist.findWells(eq,PE=False,responseYears=forecastYears,verbose=verb)
# Isolate wells included only for forecast
currentWellsDF=selectedWellsDF[selectedWellsDF['EncompassingDay']<0.].reset_index(drop=True)
forecastWellsDF=selectedWellsDF[selectedWellsDF['EncompassingDay']>=0.].reset_index(drop=True)
print(selectedWellsDF.shape,currentWellsDF.shape,forecastWellsDF.shape)

 gistMC.findWells: iw= 0  of  2863  start date is  1997-07-08
 gistMC.findWells injectionDays 10042
 gistMC.findWells: iw= 1  of  2863  start date is  nan
 gistMC.findWells injectionDays nan
 gistMC.findWells: iw= 2  of  2863  start date is  1991-01-27
 gistMC.findWells injectionDays 12396
 gistMC.findWells: iw= 3  of  2863  start date is  1952-07-22
 gistMC.findWells injectionDays 26464
 gistMC.findWells: iw= 4  of  2863  start date is  2001-04-16
 gistMC.findWells injectionDays 8664
 gistMC.findWells: iw= 5  of  2863  start date is  1976-01-25
 gistMC.findWells injectionDays 17877
 gistMC.findWells: iw= 6  of  2863  start date is  nan
 gistMC.findWells injectionDays nan
 gistMC.findWells: iw= 7  of  2863  start date is  1977-11-06
 gistMC.findWells injectionDays 17226
 gistMC.findWells: iw= 8  of  2863  start date is  1974-02-01
 gistMC.findWells injectionDays 18600
 gistMC.findWells: iw= 9  of  2863  start date is  1980-06-27
 gistMC.findWells injectionDays 16262
 gistMC.findWells: 

In [0]:
scenarioDF=gist.runPressureScenarios(eq,currentWellsDF,injDF,verbose=1)

 gistMC.runPressureScenarios: Number of wells considered:  2278
 gistMC.pressureScenario: well  0  of  2278 :  1000054
 gistMC.pressureScenario: well  10  of  2278 :  1001136
 gistMC.pressureScenario: well  20  of  2278 :  1001500
 gistMC.pressureScenario: well  30  of  2278 :  1001653
 gistMC.pressureScenario: well  40  of  2278 :  1001856
 gistMC.pressureScenario: well  50  of  2278 :  1002156
 gistMC.pressureScenario: well  60  of  2278 :  1002506
 gistMC.pressureScenario: well  70  of  2278 :  1003142
 gistMC.pressureScenario: well  80  of  2278 :  1003864
 gistMC.pressureScenario: well  90  of  2278 :  1001708
 gistMC.pressureScenario: well  100  of  2278 :  1004857
 gistMC.pressureScenario: well  110  of  2278 :  1001048
 gistMC.pressureScenario: well  120  of  2278 :  1001480
 gistMC.pressureScenario: well  130  of  2278 :  1006336
 gistMC.pressureScenario: well  140  of  2278 :  1006679
 gistMC.pressureScenario: well  150  of  2278 :  1007297
 gistMC.pressureScenario: well  160

In [0]:
filteredDF,orderedWellList=gi.summarizePPResults(scenarioDF,currentWellsDF,threshold=dPCutoff,nOrder=nWells,verbose=2)

88  disaggregationPlotPP: wells have a  1.0  psi pressure contribution in one scenario
 disaggregationPlotPP:  88  sorted
 disaggregationPlotPP:  2190  minimally-contributing wells sorted


In [0]:
disaggregationDF=gi.prepDisaggregationPlot(filteredDF,orderedWellList,jitter=0.1,verbose=0)

In [0]:
diffRange=(min(gist.diffPPVec),max(gist.diffPPVec))
rtDF,mergedWellsDF = gi.prepRTPlot(selectedWellsDF,ignoredWellsDF,minYear,diffRange,eq,clipYear=False,verbose=1)

 prepRTPlot: years before earthquake to plot: -45.01026694045174
 prepRTPlot: # of must include wells : 5
 prepRTPlot: # of wells to add to forecast : 188
 prepRTPlot: # of wells with 0 BBL disposal : 1482


In [0]:
winWellsDF,winInjDF=gi.getWinWells(filteredDF,currentWellsDF,injDF)
display(winWellsDF)

index,ID,InjectionWellId,UniqueWellIdentifier,UICNumber,APINumber,LeaseName,Operator,OperatorType,OperatorPrincipalCompany,OperatorPrincipalCompanyType,WellName,WellNumber,State,Basin,County,District,SRAOrSIR,B3InjectionType,B3InjectionStatus,RegulatoryInjectionType,PermittedMaxLiquidBPD,PermittedMaxLiquidPSIG,PermittedMaxGasMCFPerDay,PermittedMaxGasPSIG,PermittedCommercialStatus,PermittedIntervalTopFt,PermittedIntervalBottomFt,InjectionClass,PermitStage,PermittedWellDepthClassification,DaysApplicationHasBeenInReview,DaysToPermitApproval,PermitIsAmendment,PendingApplicationIsAmendment,PendingApplicationRequestedCommercialStatus,PendingApplicationRequestedIntervalTopFt,PendingApplicationRequestedIntervalBottomFt,PendingApplicationRequestedMaxLiquidBPD,PendingApplicationRequestedMaxLiquidPSIG,PendingApplicationRequestedMaxGasMCFPerDay,PendingApplicationRequestedMaxGasPSIG,PendingApplicationRequestedOperator,PendingApplicationRequestedOperatorType,PendingApplicationRequestedOperatorPrincipalCompany,PendingApplicationRequestedOperatorPrincipalCompanyType,PendingApplicationRequestedWellDepthClassification,CompletedWellDepthClassification,CompletionAndDrillingPermitStatus,StartDate,TotalVerticalDepthFt,MeasuredDepthFt,WellboreOrientation,IsOpenHole,SurfaceHoleElevationFt,SurfaceHoleLatitude,SurfaceHoleLongitude,SurfaceHoleGeographySource,SurveyLinesDescription,B3RecordAddedUTCDateTime,B3RecordUpdatedUTCDateTime,B3RecordDeletedUTCDateTime,Distances,DXs,DYs,DDRatio,YearsInjecting,EncompassingDay,EncompassingDiffusivity,EventID,TotalBBL
1084,2081279,81279,10142010000081279,84368,42-227-34375,GUITAR ESTATE -B-,AMERICO ENERGY RESOURCES,E&P,AMERICO ENERGY RESOURCES,E&P,GUITAR ESTATE -B- 2D,2D,TX,Permian - Midland,Howard,08,"Knott 15km SIR, Stanton 15km SIR",Saltwater Disposal,Active - Liquid,Injection Into Productive Zone,5000.0,2200.0,0.0,null,Non Commercial,8700.0,10800.0,Class 2 - Injection of fluids associated with oil and natural gas production,Not in Permitting Process,Permian Deep,null,null,true,null,null,null,null,null,null,null,null,null,null,null,null,null,Permian Deep,Completed,1996-06-24,11000.0,10700.0,Vertical,false,2516.0,32.274845,-101.62872,Lat/Long from Well Surface Location,660' FNL/1980' FWL,2024-02-16 20:41:47,2024-12-24 12:54:35,null,15.977322131370066,-15.891260338095195,1.6561009869561005,0.050874847681684036,28.531143052703626,-10395.0,0.022561864179013104,texnet2025agkw,1.4927081999999993E7
1093,2083417,83417,10142010000083417,86626,42-227-30394,EAST VEALMOOR DISPOSL UNIT NO.1,SURGE OPERATING,E&P,SURGE OPERATING,E&P,EAST VEALMOOR DISPOSL UNIT NO.1 2,2,TX,Permian - Midland,Howard,08,Knott 25km SIR,Saltwater Disposal,Active - Liquid,Injection Into Productive Zone,15000.0,3900.0,0.0,null,Non Commercial,7820.0,8800.0,Class 2 - Injection of fluids associated with oil and natural gas production,Not in Permitting Process,Permian Deep,null,null,true,null,null,null,null,null,null,null,null,null,null,null,null,null,Permian Deep,Completed,2005-05-20,10000.0,8800.0,Vertical,true,null,32.468334,-101.418144,Lat/Long from Well Surface Location,457' FSLL/2017' FELL,2024-02-16 20:41:47,2024-12-24 12:54:35,null,40.81568411741115,-35.65499070371125,-19.865648214805617,0.15669372987562363,19.627652292950035,-6993.0,0.2140286802069013,texnet2025agkw,2.4896833E7
1102,2084091,84091,10142010000084091,86957,42-227-34926,EAST VEALMOOR DISPOSL UNIT NO.1,SURGE OPERATING,E&P,SURGE OPERATING,E&P,EAST VEALMOOR DISPOSL UNIT NO.1 1D,1D,TX,Permian - Midland,Howard,08,Knott 25km SIR,Saltwater Disposal,Active - Liquid,Injection Into Productive Zone,15000.0,3800.0,0.0,null,Non Commercial,7600.0,9200.0,Class 2 - Injection of fluids associated with oil and natural gas production,Not in Permitting Process,Permian Deep,null,101.0,true,null,null,null,null,null,null,null,null,null,null,null,null,null,Permian Deep,Completed,1998-12-01,9500.0,9200.0,Vertical,false,null,32.477036,-101.42982,Lat/Long from Well Surface Location,"859 FNL, 467 FWL",2024-02-16 20

In [0]:

scenarioTSRDF,dPTimeSeriesR,wellIDsR,dayVecR = gist.runPressureScenariosTimeSeries(eq,winWellsDF,winInjDF,verbose=verb)

runPressureScenariosTimeSeries time axis information - nt: 1464 ; ot: 5520.0 ; dt: 10.0  earthquake index:  1457
runPressureScenariosTimeSeries r2 min/max:  7833733.589149127 2566029044.357542
runPressureScenariosTimeSeries TSOver4TT min/max:  0.028679479963385778 39.8316437346246
runPressureScenariosTimeSeries ppp min/max:  224667.40550850454 102209154707.54884
runPressureScenariosTimeSeries durations min/max:  864000.0 1264896000.0
runPressureScenariosTimeSeries epp min/max:  0.0 8.058841296979432
runPRessureScenariosTimeSeries time step  10  of  1464
runPRessureScenariosTimeSeries time step  20  of  1464
runPRessureScenariosTimeSeries time step  30  of  1464
runPRessureScenariosTimeSeries time step  40  of  1464
runPRessureScenariosTimeSeries time step  50  of  1464
runPRessureScenariosTimeSeries time step  60  of  1464
runPRessureScenariosTimeSeries time step  70  of  1464
runPRessureScenariosTimeSeries time step  80  of  1464
runPRessureScenariosTimeSeries time step  90  of  1464


In [0]:
totalPPQuantilesDF=gi.prepTotalPressureTimeSeriesPlot(dPTimeSeriesR,dayVecR,nQuantiles=21,epoch=pd.to_datetime('1970-01-01'),verbose=1)
totalPPSpaghettiDF=gi.prepTotalPressureTimeSeriesSpaghettiPlot(dPTimeSeriesR,dayVecR,gist.diffPPVec,epoch=pd.to_datetime('1970-01-01'),verbose=1)
print(totalPPQuantilesDF)

prepTotalPressureTimeSeriesPlot: deltaPP.shape= (88, 200, 1464)  dayVec.shape= (1464,)
prepTotalPressureTimeSeriesPlot: totalDeltaPP.shape= (200, 1464)
prepTotalPressureTimeSeriesPlot: quantiles: [0.0, 5.0, 10.1, 15.1, 20.1, 25.1, 30.2, 35.2, 40.2, 45.2, 50.3, 54.8, 59.8, 64.8, 69.8, 74.9, 79.9, 84.9, 89.9, 95.0, 100.0]
prepTotalPressureTimeSeriesSpaghettiPlot: deltaPP.shape= (88, 200, 1464)  dayVec.shape= (1464,)
prepTotalPressureTimeSeriesSpaghettiPlot: totalDeltaPP.shape= (200, 1464)
        DeltaPressure     Days  Realization  Percentile  Ordering       Date
0        0.000000e+00   5520.0            0         0.0       0.0 1985-02-11
1        0.000000e+00   5530.0            0         0.0       0.0 1985-02-21
2        0.000000e+00   5540.0            0         0.0       0.0 1985-03-03
48       7.144209e-11   6000.0            0        45.2      90.0 1986-06-06
49       1.114319e-10   6010.0            0        45.2      90.0 1986-06-16
...               ...      ...          ...   

In [0]:
allPPQuantilesDF,allPPSpaghettiDF=gi.getPerWellPressureTimeSeriesSpaghettiAndQuantiles(dPTimeSeriesR,dayVecR,gist.diffPPVec,wellIDsR,nQuantiles=11,epoch=pd.to_datetime('01-01-1970'))

In [0]:
wellPressureDict=gi.prepPressureAndDisposalTimeSeriesPlots(allPPQuantilesDF,allPPSpaghettiDF,winWellsDF,winInjDF,orderedWellList[:-1],verbose=0)

In [0]:
# Calculate sensitivities to different parameters
sensitivityDF,sensitivitySumDF = gist.getPressureSensitivity(winInjDF,winWellsDF,eq,verbose=2)
# I'm not getting negative pressures anymore
print(sensitivitySumDF)

getPressureSensitivity: ntaS min/max:  0.0009 0.0011
getPressureSensitivity: phiS min/max:  3.0 15.0
getPressureSensitivity: hMS min/max:  213.36 609.6
getPressureSensitivity: alphavS min/max:  5.6e-10 1.07e-09
getPressureSensitivity: betaS min/max:  3.59e-10 3.61e-10
getPressureSensitivity: kMDS min/max:  3.0 1000.0
getPressureSensitivity: SS:  [[0.00239129]
 [0.00245108]
 [0.00251086]
 [0.00245108]
 [0.00245108]
 [0.00245108]
 [0.0023617 ]
 [0.00245108]
 [0.00254045]
 [0.00127093]
 [0.00245108]
 [0.00363122]
 [0.00456122]
 [0.00245108]
 [0.00245108]
 [0.0024507 ]
 [0.00245108]
 [0.00245145]
 [0.00245108]
 [0.00245108]
 [0.00245108]]
getPressureSensitivity: TS:  [[2.00412068e-03]
 [2.05422370e-03]
 [2.10432672e-03]
 [2.28247078e-03]
 [2.05422370e-03]
 [1.86747609e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [1.06515303e-03]
 [2.05422370e-03]
 [3.04329437e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [2.05422370e-03]
 [1.

In [0]:
# This is the date to calcaulte THdPdT0
futureEQ=eq.copy()
futureEQ.loc['Origin Date']=pd.to_datetime('2026-9-1')
print(eq,futureEQ)
rateDF = gist.getTHdPdT0(winWellsDF,winInjDF,eq,futureEQ,2)

SeismicEventId                                     2743652
DataSource                       TexNet Earthquake Catalog
DataSourceUrl                                         None
EventID                                     texnet2025agkw
EventTimeUtc                           2025-01-04 12:16:53
EventTimeInLocalTimeZone              2025-01-04T06:16:53Z
EventTimeZone                                          CST
EventType                                       Earthquake
DepthKm                                           7.202148
DepthErrorKm                                      0.593583
Magnitude                                         1.401685
MagnitudeError                                         NaN
MagnitudeType                                   ml(texnet)
Location                                     Western Texas
Status                                               final
Latitude                                         32.289734
LatitudeError                                     0.2832

/Workspace/Users/bill.curry@exxonmobil.com/GIST/lib/gistMC.py:2201: RuntimeWarning: divide by zero encountered in divide
  Q_new=(dpdt * futureInjSec / OneOver4piKappaH) / expTerm


###Output

In [0]:
gist.writeRealizations(initialRunIntervalPath+'PorePressureRealizations.csv')

In [0]:
scenarioDF.to_csv(initialRunIntervalPath+'scenarios.csv')

In [0]:
filteredDF.to_csv(initialRunIntervalPath+'filteredScenarios.csv')
pd.Series(data=orderedWellList).to_csv(initialRunIntervalPath+'wellOrder.csv')

In [0]:
mergedWellsDF.to_csv(initialRunIntervalPath+'RTwells.csv')
rtDF.to_csv(initialRunIntervalPath+'RTDF.csv')

In [0]:
disaggregationDF.to_csv(initialRunIntervalPath+'disaggregation.csv')

In [0]:
totalPPQuantilesDF.to_csv(initialRunIntervalPath+'totalPPQuantiles.csv')
totalPPSpaghettiDF.to_csv(initialRunIntervalPath+'totalPPSpaghetti.csv')

In [0]:
i=0
for wellDictKey, wellDictValue in wellPressureDict.items():
  wellID=wellDictValue['WellInfo']['ID'].to_list()[0]
  wellFilePrefix='/perWell/well_'+str(i)+'_'
  wellDictValue['PPQuantiles'].to_csv(initialRunIntervalPath+wellFilePrefix+'PPQuantiles.csv')
  wellDictValue['Disposal'].to_csv(initialRunIntervalPath+wellFilePrefix+'Disposal.csv')
  wellDictValue['Spaghetti'].to_csv(initialRunIntervalPath+wellFilePrefix+'Spaghetti.csv')
  wellDictValue['WellInfo'].to_csv(initialRunIntervalPath+wellFilePrefix+'WellInfo.csv')
  print('well',i,', ID:',wellID,' completed')
  i=i+1

well 0 , ID: 2121203  completed
well 1 , ID: 2116501  completed
well 2 , ID: 2112058  completed
well 3 , ID: 2121715  completed
well 4 , ID: 2117633  completed
well 5 , ID: 2121195  completed
well 6 , ID: 2112104  completed
well 7 , ID: 2123953  completed
well 8 , ID: 2111946  completed
well 9 , ID: 2110592  completed
well 10 , ID: 2118517  completed
well 11 , ID: 2117616  completed
well 12 , ID: 2123938  completed
well 13 , ID: 2112198  completed
well 14 , ID: 2119704  completed
well 15 , ID: 2121097  completed
well 16 , ID: 2117806  completed
well 17 , ID: 2113042  completed
well 18 , ID: 2121271  completed
well 19 , ID: 2111922  completed
well 20 , ID: 2112288  completed
well 21 , ID: 2081404  completed
well 22 , ID: 2120964  completed
well 23 , ID: 2120980  completed
well 24 , ID: 2120615  completed
well 25 , ID: 2111358  completed
well 26 , ID: 2110718  completed
well 27 , ID: 2121230  completed
well 28 , ID: 2121094  completed
well 29 , ID: 2120827  completed
well 30 , ID: 212045

In [0]:
scenarioTSRDF.to_csv(initialRunIntervalPath+'materialScenariosR.csv')
np.savez_compressed(initialRunIntervalPath+'timeSeriesR.npz', deltaPP=dPTimeSeriesR,dayVec=dayVecR,wellIDs=wellIDsR)

In [0]:
sensitivityDF.to_csv(initialRunIntervalPath+'sensitivity.csv')
sensitivitySumDF.to_csv(initialRunIntervalPath+'sensitivitySum.csv')

In [0]:
rateDF.to_csv(initialRunIntervalPath+'rates.csv')

#3. Correct Data

##3.1 Export Disposal Data

In [0]:
selectedWellsDF.to_csv(disposalPath+'selectedWells.csv')
ignoredWellsDF.to_csv(disposalPath+'ignoredWells.csv')
allWellsDF=pd.concat([selectedWellsDF,ignoredWellsDF])
allWellsDF.to_csv(disposalPath+'allInZoneWells.csv')
injDF.to_csv(disposalPath+'inj.csv')
injDF.to_csv(disposalPath+'inj.zip',compression='zip')

In [0]:
# End Year of time series forecast - I don't think that I need this now.
#EndYear=2030
# Rerun select wells with an updated earthquake time set to the last day of EndYear
#forecastEQ=eq.copy()
#forecastEQ['Origin Date']=pd.to_datetime('12-31-'+str(EndYear))
#selectedForecastWellsDF,ignoredForecastWellsDF,injDF=gist.findWells(forecastEQ,PE=False,verbose=verb)
##selectedWellsDF.to_csv(disposalPath+'forecastSelectedWells.csv')
#ignoredWellsDF.to_csv(disposalPath+'forecastIgnoredWells.csv')
#allWellsDF=pd.concat([selectedWellsDF,ignoredWellsDF])
#allWellsDF.to_csv(disposalPath+'forecastAllInZoneWells.csv')
#injDF.to_csv(disposalPath+'forecastPastInj.csv')

##3.2 Import Updated Disposal Data

In [0]:
# I need to merge the updated disposal information with the prior stuff
# Load updated injection file
# Well comparisons
# Get list of well IDs in the updated injection file
# Get list of well IDs in updated well file
# Check to see overlap in well IDs
#    updated vs. prior selected wells
#    updated vs. prior ignored wells
# FilteredIgnored =  prior ignored wells - (wells in updated file and prior ignored wells)
# Merge FilteredIgnored with with updated wells
# Load injection
#    Check for time sampling of injection - all wells must be the same
#    Get overall time vector - convert to days
#    Loop over wells in well file
#      Get all disposal data for well ID
#      Convert date values to days
#      
#mergedInjFile=